<a href="https://colab.research.google.com/github/martatolos/eae-dsaa-2025/blob/main/unsupervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsupervised Algorithms

> Goal of the session:
>
> - At the end of this activity, you will understand the basics of unsupervised algorithms and how to apply them. Also see their limitations and how some approaches try to overcome them.
>
> Scope of the session
>
> - Understand the motivation and use cases of unsupervised algorithms.
> - Prepare a dataset for unsupervised learning.
> - Use KMeans clustering to group data points.
> - Analyze how KMeans clustering works.
> - Go through the limitations of KMeans clustering.
> - See metrics and methodologies to try to evaluate whether clustering is effective.
> - Use methods such as Spectral Clustering cluster data points.

### Unsupervised learning: recap theory

**Describe in your own words: what is unsupervised learning?**


<details>
  <summary>💡 Show solution</summary>
  <ul>
    <li>Unsupervised algorithms are those whose training data consists of a set of input variables $X$ without a target variable $Y$.</li>
    <li>=> We have no labels for our data points</li>
 </ul>
</details>


The main categories of unsupervised learning are:

**Clustering**
- Discovering groups with similar features within a dataset
- Grouping data points into clusters based on similarity
- Examples: K-Means, DBSCAN, Hierarchical Clustering

**Dimensionality reduction**
- Reducing the number of variables/features while preserving important information
- This reduction will allow the visualization as well as a better knowledge about your data
- Examples: PCA (Principal Component Analysis), t-SNE, UMAP

**Anomaly Detection**
- Identifying rare or unusual data points that differ significantly from the majority.
- Examples: One-Class SVM, Isolation Forest, Autoencoders (unsupervised setup)

**Association Rule Learning**
- Discovering interesting relationships (rules) between variables in large datasets
- Example: Apriori, Eclat


#### Applications

* **Customer segmentation**: the market is divided into smaller segments of buyers who have different needs, characteristics and behaviors to apply different strategies.

* **Fraud detection**: identify which transactions can be considered false pretenses. It is about finding anomalous behaviours which are not related to the general behaviour of the rest of the population.

> Kaggle example: http://archive.ics.uci.edu/ml/datasets/statlog+(australian+credit+approval)

* **Face detection using PCA**: principal component analysis is used to reduce the number of variables. The data is compressed in such a way that the main characteristics are preserved. In the case of an image where a face appears, we know that not all the pixels represent the main features of the face. Using PCA, we extract the main ones which define a face and reduce dimensions.  
> PCA example from scratch to detect faces: [implementation example](https://medium.com/@reubenrochesingh/building-face-detector-using-principal-component-analysis-pca-from-scratch-in-python-1e57369b8fc5)

### Unsupervised learning in Python

**Dependencies**

- ``numpy`` 2.0.2
- ``nbformat``
- ``pandas`` 2.2.2
- ``plotly`` 5.24.1
- ``scikit-learn`` 1.6.1

In [ ]:
%pip install numpy==2.0.2 nbformat pandas==2.2.2 plotly==5.24.1 scikit-learn==1.6.1

**Imports**

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.datasets import make_blobs, make_circles
from sklearn.metrics import pairwise_distances_argmin, silhouette_score

### K-means clustering

*K-means* is an unsupervised algorithm from *clustering* category. Its purpose is to partition a set of $n$ observations / objects (e.g., images or attributes of customers) into $k$ groups where each observation belongs to the group whose **mean value is the closest**.

**Data generation**

First, we'll generate example data for illustrating the algorithm. For this, we create a two dimensional example using one of the `scikit-learn` component called `make\_blobs` to generate clusters.

In [ ]:
# Generate 300 data points belonging to 3 different clusters
n_clusters = 3
X, y = make_blobs(n_samples=300, centers=n_clusters, cluster_std=0.50, random_state=0)

In [ ]:
# Plot the generated data
fig = px.scatter(x=X[:, 0], y=X[:, 1], size_max=15)
fig.show()

We can inspect the documentation of the component using the `help()` function or refer to the [website](https://scikit-learn.org/0.15/modules/generated/sklearn.datasets.make_blobs.html).

In [ ]:
# Print the documentation
help(make_blobs)

We can see that the number of groups are 3. The ${K-means}$ algorithm should detect automatically to which group each data point has to be assigned.

**Applying K-Means**

For running the algorithm we have instantiate an instance of `KMeans` class and run the `fit` method:

In [ ]:
# Create kmeans instance and fit the model to the data
kmeans = KMeans(n_clusters=n_clusters)  # Number of groups are pre-defined
kmeans.fit(X)  # Training

We can use the model to predict the cluster / group to which a data point belongs to

In [ ]:
# Predict the cluster for data point
y_predicted = kmeans.predict(X)  # Prediction
y_predicted

Each of the 300 points has been associated with one of the three previously set groups. We could apply the same to new data points to predict the group to which they belong.

**Visualize the result k-means works**

For representing clusters, K-Means stores the center centroids (i.e., the mean vector of all points assigned to the cluster).

In [ ]:
centers = kmeans.cluster_centers_
print(centers)

We plot the points with the colour associated to their specific group. We also plot the centroids groups which are defined as the minimum mean distance of each set.

In [ ]:
# Create a figure
fig = go.Figure()

# Add data points colored by their cluster labels
fig.add_trace(
    go.Scatter(
        x=X[:, 0],
        y=X[:, 1],
        mode="markers",
        marker={"color": y_predicted, "colorscale": "viridis", "size": 10},
        name="Data Points",
    )
)

# Add cluster centers
fig.add_trace(
    go.Scatter(
        x=centers[:, 0],
        y=centers[:, 1],
        mode="markers",
        marker={"color": "red", "size": 12, "symbol": "x"},
        name="Cluster Centers",
    )
)

# Update layout
fig.update_layout(title="K-means Clustering Results", xaxis_title="Feature 1", yaxis_title="Feature 2")

fig.show()

### How K-Means work

**Describe in your own words: how does K-Means work?**


<details>
  <summary>💡 Show solution</summary>
    This method is based on the **Expectation-Maximization algorithm** and the approach consists of the following steps:

    1. Initial estimation of the centroids.
    2. *Expectation step*: Assign the points to the closest cluster.
    3. *Maximization step*: Set the centroids based on the new computed mean.
    4. Go back to step 2 if the centroids have changed. Otherwise, stop.

    When the centroids are not changing, the algorithm has converged.
</details>



See the following code, the first function implements the same process we run with ``scikit-learn`` and the second allows to deep dive into the algorithm to see how it works though visualizations:

In [ ]:
def find_clusters(
    X: np.ndarray[np.float64],
    centers: np.ndarray[np.float64],
    max_iters: int,
) -> tuple[np.ndarray[np.float64], np.ndarray[np.float64], int]:
    """Find the clusters within a dataset.

    :param X: Dataset with samples to be clustered.
    :param centers: Initial centers of the clusters.
    :param max_iters: Maximum number of iterations.
    :return: Tuple with the centers and labels for each iteration and the number of iterations.
    """
    # Initial parameters
    iters = 0
    n_clusters = len(centers)  # Number of clusters
    centers_iters = []  # Save centers for each iteration
    labels_iters = []  # Save assignments for each iteration

    for i in range(max_iters):
        # Compute the Euclidean distance from each point to each centroid
        distances = np.linalg.norm(X[:, None] - centers, axis=2)

        # Assign each point to the closet the center based on the computed distances
        labels = np.argmin(distances, axis=1)

        # Save results
        centers_iters.append(centers)
        labels_iters.append(labels)

        # Reallocate the centroids
        new_centers = np.array([X[labels == i].mean(0) for i in range(n_clusters)])

        # Check convergence
        # In this case we're forcing the function to reach the same center as the cluster
        if np.all(centers == new_centers):
            break

        centers = new_centers
        iters += 1

    # The output lists are converted to numpy arrays.
    return np.array(centers_iters, dtype=np.float64), np.array(labels_iters, dtype=np.float64), iters

In [ ]:
def visualize_kmeans_process(centers_iters: np.ndarray, labels_iters: np.ndarray, n_clusters: int, iters: int) -> None:
    """
    Visualize the k-means process for each iteration using Plotly.

    :param centers_iters: Centers for each iteration
    :param labels_iters: Assignments for each iteration
    :param n_clusters: Number of clusters
    :param iters: Number of iterations until convergence.
    """
    n_plots = iters + 1
    n_cols = min(3, n_plots)
    n_rows = int(np.ceil(n_plots / n_cols))

    # Determine figure width and height based on number of columns and rows
    width = n_cols * 600
    height = n_rows * 400

    # Create subplots with computed number of rows and columns
    fig = make_subplots(rows=n_rows, cols=n_cols, subplot_titles=[f"Iteration {i}" for i in range(n_plots)])

    for i in range(n_plots):
        row = i // n_cols + 1
        col = i % n_cols + 1

        fig.add_trace(
            go.Scatter(
                x=X[:, 0],
                y=X[:, 1],
                mode="markers",
                marker={"color": labels_iters[i], "colorscale": "viridis", "size": 8},
                showlegend=False,
            ),
            row=row,
            col=col,
        )

        for cluster in range(n_clusters):
            fig.add_trace(
                go.Scatter(
                    x=[centers_iters[i][cluster][0]],
                    y=[centers_iters[i][cluster][1]],
                    mode="markers",
                    marker={"color": "red", "size": 10, "symbol": "x"},
                    showlegend=False,
                ),
                row=row,
                col=col,
            )

    fig.update_layout(title="K-means Clustering Process", width=width, height=height)
    fig.show()

In [ ]:
centers = np.array([[1, 1], [2, 3], [2, 1]])
#centers = np.array([[1, 1], [1, 3], [2, 1]])

centers_iters, labels_iters, iters = find_clusters(X, centers, 10)

n_clusters = len(centers)
visualize_kmeans_process(centers_iters, labels_iters, n_clusters, iters)

The first graph shows an initial cluster assignment that is not the desired one because of the random centroids used. However, the centroids are getting closer to their corresponding groups until the solution coverges. **It happens when the distance of the points to the closest centroid does not produce new assigments**

**Exercise: Run the algorithm using different starting points. Investigate the obtained results and the algorithm behaviour (e.g., after how many iterations does the solution converges)**

**Does the result depend on the initial centroids?**

In [ ]:
centers = np.array([[2, 0], [3, 1], [2, 1]])

centers_iters, labels_iters, iters = find_clusters(X, centers, 20)

n_clusters = len(centers)
visualize_kmeans_process(centers_iters, labels_iters, n_clusters, iters)

In [ ]:
centers = np.array([[1, 1], [2, 3]])

centers_iters, labels_iters, iters = find_clusters(X, centers)

n_clusters = len(centers)
visualize_kmeans_process(centers_iters, labels_iters, n_clusters, iters)

### Limitations of K-means

**Limitation 1: Number of clusters**

One of the most important limitations is that $K-means$ needs the number of groups as an argument. How are we going to know a priori the number of groups if we want to use this method to figure it out?

What happen if we had chosen a different number of clusters?

In [ ]:
kmeans = KMeans(n_clusters=5)  # Set number of clusters
kmeans.fit(X)  # Training
y_kmeans = kmeans.predict(X)  # Prediction
centers = kmeans.cluster_centers_

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=X[:, 0],
        y=X[:, 1],
        mode="markers",
        marker={"color": y_kmeans, "colorscale": "Viridis", "size": 10},
        name="Data Points",
    )
)
fig.add_trace(
    go.Scatter(
        x=centers[:, 0],
        y=centers[:, 1],
        mode="markers",
        marker={"color": "red", "symbol": "x", "size": 12},
        name="Cluster Centers",
    )
)
fig.show()

To solve this problem, we can execute multiple $K-means$ with different number of groups (i.e., with different values for `k`) and choose the one which meets a certain criteria. There are several criteria that allow us to measure "how well" the clusters have achieved. The two most famous criteria are the Elbow and the Silhouette methods.

_**Elbow Method**_
- Run $K-Means$ for a range of $k$ values (e.g., 1 to 10).
- For each k, compute the Within-Cluster Sum of Squares (WCSS) — the sum of squared distances between points and their cluster centroids.
- Plot `k` vs. WCSS.
- The “elbow” point in the graph (where WCSS starts to decrease more slowly) suggests the best value for k.

Let's inspect the Elbow diagram for the given example.

In [ ]:
# Store WCSS for different values of k
from matplotlib import pyplot as plt

wcss = []
K_range = range(1, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)  # inertia_ is the WCSS

# Plot Elbow Graph
plt.plot(K_range, wcss, marker='o')
plt.title("Elbow Method for Optimal k")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("WCSS (Inertia)")
plt.xticks(K_range)
plt.grid(True)
plt.show()

_**Silhouette**_
- Measures how similar an object is to its own cluster compared to other clusters.
- It provides a combined measure of _cohesion_ and _separation_
- _Cohesion_: How close each point in a cluster is to other points in the same cluster
- _Separation_: How far apart a point is from points in other clusters

How it works:
- For each data point $i$:
1. $a(i)$: average distance from i to all other points in the same cluster (intra-cluster distance).
2. $b(i)$: lowest average distance from i to all points in any other cluster (nearest-cluster distance).
3. Then compute the silhouette score for point by: $s(i) = b(i) - a(i) / max(a(i), b(i))$
> $𝑠(𝑖) ≈ 1$: point is well clustered. <br />
> $𝑠(𝑖) ≈ 0$: point lies between clusters. <br />
> $𝑠(𝑖) < 0:$ point might be in the wrong cluster.
4. The overall silhouette score is the average of all $s(i)$ values in the dataset.

In [ ]:
scores = []
groups = np.arange(2, 11)  # 2, 3, 4, ..., 8, 9, 10

for k in groups:
    kmeans = KMeans(n_clusters=k, n_init=10).fit(X)
    labels = kmeans.labels_
    scores.append(silhouette_score(X, labels, metric="euclidean"))

# Create a figure using Plotly
fig = go.Figure()

# Add a line trace for silhouette scores
fig.add_trace(go.Scatter(x=groups, y=scores, mode="lines+markers"))
fig.update_layout(
    title="Silhouette Scores for Different Numbers of Clusters",
    xaxis_title="Number of Clusters",
    yaxis_title="Silhouette Score",
    xaxis={"tickmode": "linear"},
)

fig.show()

In general, the following guideline help to interpret silhouette scores:

| Score Range | Interpretation                            |
| ----------- | ----------------------------------------- |
| 0.71 – 1.00 | Strong structure, clear clusters          |
| 0.51 – 0.70 | Reasonable structure                      |
| 0.26 – 0.50 | Weak structure, overlapping clusters      |
| < 0.25      | No substantial structure, poor clustering |

The silhouette score is often favoured since:
- Works with any distance metric (e.g., Euclidean, Manhattan).
- Helps choose the optimal number of clusters (k) programmatically by comparing average silhouette scores for different values.


**Limitation 2: Linear separation**

The fundamental model assumptions of k-means (points will be closer to their own cluster center than to others) means that the algorithm will often be ineffective if the clusters have complicated geometries. In particular, the boundaries between k-means clusters will always be linear, which means that it will fail for more complicated boundaries. Consider the following data, along with the cluster labels found by the typical k-means approach:

In [ ]:
X, y = make_circles(n_samples=400, factor=0.3, noise=0.05)
fig = px.scatter(
    x=X[:, 0],
    y=X[:, 1],
    color=y.astype(str),
    title="Circles Dataset - Instance Classes",
    labels={"x": "Feature 1", "y": "Feature 2", "color": "Class"},
)
fig.update_traces(marker={"size": 10})
fig.show()

**Question: Do the groups above have a linear separation?**

In [ ]:
kmeans = KMeans(n_clusters=2).fit(X)
y_kmeans = kmeans.predict(X)

In [ ]:
centers = kmeans.cluster_centers_

fig = go.Figure()

# Add data points colored by their cluster labels
fig.add_trace(
    go.Scatter(
        x=X[:, 0],
        y=X[:, 1],
        mode="markers",
        marker={"color": y_kmeans, "colorscale": "viridis", "size": 10},
        name="Data Points",
    )
)

# Add cluster centers
fig.add_trace(
    go.Scatter(
        x=centers[:, 0],
        y=centers[:, 1],
        mode="markers",
        marker={"color": "red", "size": 12, "symbol": "x"},
        name="Cluster Centers",
    )
)

# Update layout
fig.update_layout(title="K-means Clustering Results", xaxis_title="Feature 1", yaxis_title="Feature 2")

fig.show()

**One solution: Spectral clustering**

In order to solve this, we can use a kernel transformation to project the data into a higher dimension where a linear separation is possible. We might imagine using the same trick to allow k-means to discover non-linear boundaries. One version of this kernelized k-means is implemented in `scikit-learn` within the `SpectralClustering` estimator. It uses the graph of nearest neighbors to compute a higher-dimensional representation of the data, and then assigns labels using a k-means algorithm:

In [ ]:
model = SpectralClustering(n_clusters=2, affinity="nearest_neighbors", assign_labels="kmeans")
labels = model.fit_predict(X)

# Create a figure using Plotly
fig = go.Figure()

# Add data points colored by their cluster labels
fig.add_trace(
    go.Scatter(
        x=X[:, 0],
        y=X[:, 1],
        mode="markers",
        marker={"color": labels, "colorscale": "viridis", "size": 10},
        name="Data Points",
    )
)

# Update layout
fig.update_layout(title="Spectral Clustering Results", xaxis_title="Feature 1", yaxis_title="Feature 2")

fig.show()

**In real cases**, it's hard to check if your clusters have a linear separation because of the number of dimensions. The approach will be to try different models and see how it works based on your requirements.

**Limitation 3: Categorical Variables**

The standard k-means algorithm isn't directly applicable to categorical data, for various reasons. The sample space for categorical data is discrete, and doesn't have a natural origin. A Euclidean distance function on such a space isn't really meaningful.

There's a variation of k-means known as k-modes, introduced in this [paper by Zhexue Huang](http://www.cs.ust.hk/~qyang/Teaching/537/Papers/huang98extensions.pdf), which is suitable for categorical data.
An example how this can be implemented in python can be found here:
- https://towardsdatascience.com/the-k-prototype-as-clustering-algorithm-for-mixed-data-type-categorical-and-numerical-fe7c50538ebb

### Dimensionality reduction

Dimensionality reduction is the process of reducing the number of input features (dimensions) in a dataset while preserving as much relevant information as possible. It's especially useful when working with high-dimensional data, such as images, text, or sensor data.

**Why is / can dimensionality reduction be important?**


<details>
  <summary>💡 Show solution</summary>

High-dimensional data can cause several issues:
  <ul>
    <li>Computational inefficiency: More features mean longer training times.</li>
    <li>Curse of dimensionality: In high dimensions, data becomes sparse and less meaningful.</li>
    <li>Overfitting: Models may learn noise instead of signal.</li>
    <li>Redundancy: Some features may be highly correlated and not provide additional information.</li>
   </ul>

Reducing dimensions helps to:
    <ul>
        <li>Speed up machine learning algorithms.</li>
        <li>Improve model performance and generalization.</li>
        <li>Enable visualization in 2D or 3D.</li>
    </ul>
</details>


**Types of dimensionality reduction**

1. Feature Selection
- Select a subset of the original features.
- Examples: removing low-variance features, recursive feature elimination.

2. Feature Extraction (Transformation)
- Create new features that are combinations of original ones
- Examples: PCA (Principal Component Analysis) – linear combinations preserving variance, t-SNE / UMAP – nonlinear methods for visualization

**Principal Component Analysis (PCA)**

Principal Component Analysis (PCA) is a dimensionality reduction technique used to simplify large datasets by transforming them into a smaller set of variables called principal components, while preserving as much of the original data's variability as possible.

_**How does it work (sketch)**_
1. Center the data: subtract the mean of each feature to center the data around the origin.
2. Compute covariance matrix: shows how features vary with respect to each other.
3. Eigen decomposition: find eigenvectors and eigenvalues of the covariance matrix.
- Eigenvectors = directions of the new feature space (principal components).
- Eigenvalues = amount of variance carried in each direction.
4. Sort components: by eigenvalue (variance explained).
5. Project the data: Onto the top $k$ components to reduce dimensions.


**PCA in Python**

The following example shows how to apply PCA on the MNIST dataset.


In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Load MNIST data (70,000 images of 28x28 digits)
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X, y = X / 255.0, y.astype(int)  # Normalize pixel values

X = X[:2000]
y = y[:2000]

# Applying pca
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='tab10')
plt.colorbar(scatter, ticks=range(10))
plt.title("K-Means Clusters on MNIST (2D PCA)")
plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.show()
